In [1]:
# Importar as bibliotecas

from datetime import date
import json
import os
import pandas
import urllib.request

In [2]:
# Traz as opções de configuração do JSON

json_file = os.path.abspath('../../../options.json')

with open(json_file, 'r', encoding='utf-8') as j_file:
    json_data = json.load(j_file)

In [3]:
# Declaração dos caminhos

cdi_data = os.path.abspath('../../../data')

cdi_dir = os.path.join(cdi_data, 'cdi')
cdi_file = 'cdi_data.csv'
cdi_path = os.path.join(cdi_dir, cdi_file)

In [4]:
# Obtenção da planilha com os dados diários do CDI

df_cdi = pandas.read_csv(cdi_path, sep=';')

In [5]:
# Filtragem pela data dos dados

START_DATE = json_data['CONFIG']['START_DATE']
END_DATE = json_data.get("CONFIG", False).get("END_DATE", False) or str(date.today())

df_cdi = df_cdi.query("@START_DATE <= DATE <= @END_DATE").copy()

In [6]:
# Criar uma planilha com os dados de valorização

list_cdi_daily = (df_cdi['CDI_TAX']).to_list()
list_day_date = (df_cdi['DATE']).to_list()
list_cdi_daily.pop() # Valorização começa em 0

first_cdi_valuation = 0
list_cdi_valuation = [first_cdi_valuation]
last_cdi_tax_calc = 1

for cdi_tax in list_cdi_daily:
    last_cdi_tax_calc =  (1 + (cdi_tax / 100)) * last_cdi_tax_calc
    list_cdi_valuation.append(str((last_cdi_tax_calc - 1) * 100) + '%' )

In [7]:
# Geração de um '.csv' com a valorização do CDI

df_cdi_valuation = pandas.DataFrame({ 'CDI_TAX_VALUATION': list_cdi_valuation, 'DATE': list_day_date })

cvm_valuation_archive = 'cdi_valuation.csv'
cvm_valuation_path = os.path.join(cdi_dir, cvm_valuation_archive)

df_cdi_valuation.to_csv(cvm_valuation_path, sep=';', index=False)
